In [55]:
#default_exp tidb_connect

In [56]:
#export
import requests
import time

PUBLIC_KEY = '414VD1N0'
PRIVATE_KEY = 'aefec559-36eb-4cd1-b6c3-59fa74303d41'
REGION = 'ap-southeast-1'
APP_ID = 'WizQVcwy'

cluster_id = '10954456779882628977'
database = 'toyota_test'

In [57]:
#export
class Chat2QueryAPI:
    def __init__(self, public_key, private_key, region, app_id, cluster_id, database):
        self.public_key = public_key
        self.private_key = private_key
        self.region = region
        self.app_id = app_id
        self.cluster_id = cluster_id
        self.database = database
        self.base_url = f'https://{region}.data.tidbcloud.com/api/v1beta/app/chat2query-{app_id}'

    def _make_request(self, endpoint, method="get", params=None, json=None):
        url = f'{self.base_url}/{endpoint}'
        auth = requests.auth.HTTPDigestAuth(self.public_key, self.private_key)
        headers = {'Content-Type': 'application/json'}

        if method.lower() == "post":
            response = requests.post(url, auth=auth, headers=headers, json=json)
        else:
            response = requests.get(url, auth=auth, headers=headers, params=params)

        if response.status_code == 200:
            return response.json()
        else:
            return response.text

    def post_data_summary(self):
        payload = {'cluster_id': self.cluster_id, 'database': self.database}
        return self._make_request("endpoint/v2/dataSummaries", "post", json=payload)

    def get_data_summary(self):
        params = {'cluster_id': self.cluster_id, 'database': self.database}
        return self._make_request("endpoint/v2/dataSummaries", "get", params=params)

    def get_job_status(self, job_id):
        return self._make_request(f"endpoint/v2/jobs/{job_id}")

    def execute_sql(self, raw_question):
        payload = {'cluster_id': self.cluster_id, 'database': self.database, 'raw_question': raw_question}
        return self._make_request("endpoint/v2/chat2data", "post", json=payload)

    def get_sql_job_result(self, raw_question, full=False):
        execution_result = self.execute_sql(raw_question)
        job_id = execution_result.get('result', {}).get('job_id')
        
        if not job_id:
            return {"error": "Job ID not found in execution result"}
        print('Generating Result')
        time.sleep(10)  # Wait for the job to process
        job_status_result = self.get_job_status(job_id)

        if full:
            return job_status_result
        
        # Adjusting the structure to match the provided example
        job_result = job_status_result.get('result', {}).get('result', {})
        task_tree = job_result.get('task_tree', {}).get('0', {})
        
        simplified_result = {
            'clarified_task': task_tree.get('clarified_task', 'Not specified'),
            'sql_query': task_tree.get('sql', 'Not specified'),
            'columns': [col['col'] for col in task_tree.get('columns', [])] if 'columns' in task_tree else [],
            'rows': task_tree.get('rows', []) if 'rows' in task_tree else []
        }

        return simplified_result
    
def print_pretty_result(result):
    for key, value in result.items():
        if isinstance(value, list):  # For 'Columns' and 'Rows'
            print(f"{key}:")
            if key == "rows":
                for row in value:
                    print("    ", ", ".join(str(item) for item in row))
            else:
                print("    ", ", ".join(value))
        else:
            print(f"{key}: {value}")

In [61]:
api = Chat2QueryAPI('414VD1N0', 'aefec559-36eb-4cd1-b6c3-59fa74303d41', 'ap-southeast-1', 'WizQVcwy', '10954456779882628977', 'toyota_test')
print(api.post_data_summary())

{'code': 200, 'msg': '', 'result': {'data_summary_id': 279074, 'job_id': '5e50da97ca594e229018e2cfae1c9cea'}}


In [62]:
result = api.get_sql_job_result("I want to see stock of items that lower than minimum stock")
print(result)

Generating Result
{'clarified_task': 'Retrieve the stock of items that are lower than the minimum stock level.', 'sql_query': 'SELECT `part_id`, `current_stock` FROM `toyota_inventory_records` WHERE `current_stock` < `minimum_stock_level`;', 'columns': ['part_id', 'current_stock'], 'rows': [['P00005', '4'], ['P00021', '8'], ['P00025', '18'], ['P00028', '1'], ['P00031', '15'], ['P00039', '1'], ['P00057', '1'], ['P00085', '14'], ['P00089', '5'], ['P00091', '8'], ['P00096', '10'], ['P00103', '8'], ['P00107', '8'], ['P00109', '17'], ['P00112', '11'], ['P00115', '2'], ['P00116', '11'], ['P00125', '10'], ['P00126', '4'], ['P00136', '6'], ['P00161', '0'], ['P00181', '9'], ['P00187', '11'], ['P00188', '6'], ['P00190', '5'], ['P00194', '0'], ['P00195', '4'], ['P00196', '12'], ['P00211', '3'], ['P00218', '1'], ['P00220', '9'], ['P00222', '1'], ['P00223', '7'], ['P00230', '6'], ['P00240', '8'], ['P00242', '1'], ['P00243', '0'], ['P00247', '9'], ['P00251', '3'], ['P00254', '8'], ['P00261', '10'], 

In [ ]:
# def post_data_summary(cluster_id: str, database: str):
#     url = f'https://{REGION}.data.tidbcloud.com/api/v1beta/app/chat2query-{APP_ID}/endpoint/v2/dataSummaries'
#     auth = requests.auth.HTTPDigestAuth(PUBLIC_KEY, PRIVATE_KEY)
#     headers = {'Content-Type': 'application/json'}
#     payload = {
#         'cluster_id': cluster_id,
#         'database': database
#     }
    
#     response = requests.post(url, auth=auth, headers=headers, json=payload)
#     if response.status_code == 200:
#         return response.json()
#     else:
#         return response.text
    
# def get_data_summary(cluster_id: str, database: str):
#     url = f'https://{REGION}.data.tidbcloud.com/api/v1beta/app/chat2query-{APP_ID}/endpoint/v2/dataSummaries'
#     auth = requests.auth.HTTPDigestAuth(PUBLIC_KEY, PRIVATE_KEY)
#     headers = {'Content-Type': 'application/json'}
#     params = {
#         'cluster_id': cluster_id,
#         'database': database
#     }
    
#     response = requests.get(url, auth=auth, headers=headers, params=params)
#     if response.status_code == 200:
#         return response.json()
#     else:
#         return response.text


# def get_job_status(job_id: str):
#     url = f'https://{REGION}.data.tidbcloud.com/api/v1beta/app/chat2query-{APP_ID}/endpoint/v2/jobs/{job_id}'
#     auth = requests.auth.HTTPDigestAuth(PUBLIC_KEY, PRIVATE_KEY)
#     headers = {'Content-Type': 'application/json'}
    
#     response = requests.get(url, auth=auth, headers=headers)
#     if response.status_code == 200:
#         data = response.json()
#         result = data.get("result", {})

#         # If 'task_tree' is present, format the response accordingly
#         if "task_tree" in result:
#             task_tree = result.get('task_tree', {}).get('0', {})
#             task_details = {
#                 "Columns": task_tree.get('columns', []),
#                 "Rows": task_tree.get('rows', []),
#                 "SQL": task_tree.get('sql', 'Not available'),
#                 "Time Elapsed": result.get('time_elapsed', 'Not available'),
#                 "Status": result.get('status', 'Not available')
#             }
#             formatted_response = {
#                 "Code": data.get('code'),
#                 "Task Tree": task_details
#             }
#             return formatted_response
#         else:
#             # If 'task_tree' is not present, return the original 'result' part of the response
#             return {"Code": data.get('code'), "Result": result}
#     else:
#         return {"Error": response.text, "Code": response.status_code}

# # Example usage, assuming the 'cluster_id' is defined elsewhere or

    
# def execute_sql(cluster_id: str, database: str, raw_question: str):
#     url = f'https://{REGION}.data.tidbcloud.com/api/v1beta/app/chat2query-{APP_ID}/endpoint/v2/chat2data'
#     auth = requests.auth.HTTPDigestAuth(PUBLIC_KEY, PRIVATE_KEY)
#     headers = {'Content-Type': 'application/json'}
#     payload = {
#         'cluster_id': cluster_id,
#         'database': database,
#         'raw_question': raw_question
#     }
    
#     response = requests.post(url, auth=auth, headers=headers, json=payload)
#     if response.status_code == 200:
#         return response.json()
#     else:
#         return response.text
    
# def get_sql_job_result(cluster_id: str, database: str, raw_question: str, full: bool = False):
#     execution_result = execute_sql(cluster_id, database, raw_question)
#     job_id = execution_result.get('result', {}).get('job_id')
    
#     if not job_id:
#         return {"error": "Job ID not found in execution result"}
    
#     time.sleep(10)  # Wait for the job to process

#     job_status_result = get_job_status(job_id)
#     if full:
#         return job_status_result
    
#     task_tree = job_status_result.get('Result', {}).get('result', {}).get('task_tree', {}).get('0', {})
    
#     simplified_result = {
#         'Clarified Task': task_tree.get('clarified_task', 'Not specified'),
#         'SQL Query': task_tree.get('sql', 'Not specified'),
#         'Columns': [col['col'] for col in task_tree.get('columns', [])],
#         'Rows': task_tree.get('rows', [])
#     }

#     return simplified_result

# def print_pretty_result(result):
#     for key, value in result.items():
#         if isinstance(value, list):  # For 'Columns' and 'Rows'
#             print(f"{key}:")
#             if key == "Rows":
#                 # Assuming you want a tabular-like print for rows
#                 for row in value:
#                     print("    ", ", ".join(str(item) for item in row))
#             else:
#                 # Simply print the list items for 'Columns'
#                 print("    ", ", ".join(value))
#         else:
#             print(f"{key}: {value}")


In [ ]:
# Usage example

result = api.post_data_summary()
print(result)


{'code': 200, 'msg': '', 'result': {'data_summary_id': 277707, 'job_id': '1728dc3e5ca44b1797583e1806749547'}}


In [ ]:
# Example usage
job_id = '1728dc3e5ca44b1797583e1806749547'
result = api.get_job_status(job_id)
print(result)

{'code': 200, 'msg': '', 'result': {'ended_at': 1711395957, 'job_id': '1728dc3e5ca44b1797583e1806749547', 'reason': '', 'result': None, 'status': 'done'}}


In [ ]:
result = get_data_summary(cluster_id, database)
print(result)

{'code': 200, 'msg': '', 'result': {'data_summary_id': 277707, 'status': 'done'}}


In [ ]:
result = execute_sql(cluster_id, database, "I want to see the average rating in the past 10 months")
print(result)

{'code': 200, 'msg': '', 'result': {'data_summary_id': 277707, 'job_id': 'd7ad96db70b84d2d9e9f7c649fa02c1d', 'session_context_id': 277870, 'session_id': '30db9a20663d420ab396727924007d48'}}


In [ ]:
result = get_job_status('d7ad96db70b84d2d9e9f7c649fa02c1d')
print(result)

{'Code': 200, 'Result': {'ended_at': 1711351433, 'job_id': 'd7ad96db70b84d2d9e9f7c649fa02c1d', 'reason': '', 'result': {'question_id': '26db060e-9163-4f77-b6d2-493146d6f988', 'raw_question': 'I want to see the average rating in the past 10 months', 'session_context_id': 277870, 'task_tree': {'0': {'assumptions': [], 'breakdown_type': 'Resolve', 'chartOptions': {}, 'clarified_task': 'Find the average rating in the past 10 months.', 'columns': [{'col': 'average_rating'}], 'description': '', 'level': 0, 'parent_task': '', 'parent_task_id': '', 'possibleExplanations': '', 'reason': 'no proxy needed for task', 'recommendations': {}, 'rows': [['3.0676']], 'sequence_no': 0, 'sql': 'SELECT AVG(`rating`) AS `average_rating` FROM `customer_feedback` WHERE `feedback_date` >= DATE_SUB(CURDATE(), INTERVAL 10 MONTH);', 'task': 'I want to see the average rating in the past 10 months', 'task_id': '0'}}, 'time_elapsed': 7.987234354019165}, 'status': 'done'}}


In [ ]:
raw_question ='I want to see the average rating in the past 10 months by month'
result = get_sql_job_result(cluster_id, database, raw_question)

In [ ]:
print_pretty_result(result)

Clarified Task: Retrieve the average rating in the past 10 months by month.
SQL Query: SELECT MONTH(`feedback_date`) AS `month`, AVG(`rating`) AS `average_rating` FROM `customer_feedback` WHERE `feedback_date` >= DATE_SUB(CURDATE(), INTERVAL 10 MONTH) GROUP BY MONTH(`feedback_date`);
Columns:
     month, average_rating
Rows:
     12, 3.0571
     9, 3.2766
     2, 2.8372
     10, 3.0667
     7, 3.4211
     8, 3.1064
     6, 3.1923
     3, 2.7143
     11, 3.0000
     1, 2.8333
     5, 3.2000


{'Code': 200, 'Result': {'ended_at': 1711394028, 'job_id': 'd574d04888b445d3b5d132b3f8d7876f', 'reason': '', 'result': {'question_id': '967d9ed6-fbf5-4305-ad11-217f06721158', 'raw_question': 'I want to see the average rating in the past 10 months by month', 'session_context_id': 278043, 'task_tree': {'0': {'assumptions': [], 'breakdown_type': 'Resolve', 'chartOptions': {}, 'clarified_task': 'Retrieve the average rating in the past 10 months by month.', 'columns': [{'col': 'month'}, {'col': 'average_rating'}], 'description': '', 'level': 0, 'parent_task': '', 'parent_task_id': '', 'possibleExplanations': '', 'reason': 'no proxy needed for task', 'recommendations': {}, 'rows': [['2', '2.8372'], ['9', '3.2766'], ['12', '3.0571'], ['1', '2.8333'], ['11', '3.0000'], ['5', '3.2000'], ['10', '3.0667'], ['3', '2.7143'], ['7', '3.4211'], ['8', '3.1064'], ['6', '3.1923']], 'sequence_no': 0, 'sql': 'SELECT MONTH(`feedback_date`) AS `month`, AVG(`rating`) AS `average_rating` FROM `customer_feedback` WHERE `feedback_date` >= DATE_SUB(CURDATE(), INTERVAL 10 MONTH) GROUP BY MONTH(`feedback_date`);', 'task': 'I want to see the average rating in the past 10 months by month', 'task_id': '0'}}, 'time_elapsed': 5.93340539932251}, 'status': 'done'}}

In [ ]:
%md


UsageError: Line magic function `%md` not found.
